# Pre-Processing Pipeline

In [1]:
#Given pseudocode to grab the .h5 data file
import numpy as np
import h5py
from scipy import signal
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np

with h5py.File("/home/ljian/private/PHYS 139 Final Project/deepclean-1251335314-4097.h5", "r") as f:
    X = []
    for channel, timeseries in f.items():
        if channel == "H1:GDS-CALIB_STRAIN":
            y = timeseries[:]
        else:
            X.append(timeseries[:])
X = np.stack(X, axis=-1) # this will do channels last (as is typically done for Keras)
print(X.shape)
print(y.shape)

(16781312, 21)
(16781312,)


## Batching

In [ ]:
# then to construct inputs/outputs, you just
# slice windows from these two timeseries

# window_size = 8192 to replicate data size for paper
# Can change this around to try different shapes
window_length = 2
sample_rate = 4096
window_size = window_length * sample_rate

# sample a batch of random windows
# Dr. Duarte suggests making a LOT
# Like 10k
# If we want less, standard is 32
batch_size = 1000
X_batch, y_batch = [], []
for i in range(batch_size):
    idx = np.random.randint(X.shape[0] - window_size)
    X_batch.append(X[idx: idx + window_size])
    y_batch.append(y[idx: idx + window_size])
    
#can do the above process as many times as you want
#maybe change batchsize to 10000
#increasing batch_size increases chances to see overlap, 
#different data in different areas of the dataset 

X_batch = np.stack(X_batch)
y_batch = np.stack(y_batch)

print(X_batch.shape)
print(y_batch.shape)

plt.plot(y[0:100])
plt.title("Strain Data")
plt.show()

plt.plot(X_batch[0][:,0])
plt.title("First Aux Channel. First Batch")
plt.show()

strain_data = y_batch
witness_data = X_batch
channels = 21

#may be the wrong size
#Ask author what size to train model on, sample more finely doesnt have to be most 32k, try (8192, 21) slices; would
#likely be 2s of data.
#possibly increase batch size
#have to train_test_split earlier, before bringing into cnn_model

## Pre-processing data structure:

1. 8th Order Butterworth: Aliasing filter to mitigate power from outside the witnessed noise frequencies;

2. StandardScalar: Unit variance, Zero mean;

3. Windowing: Divide data into smaller overlapping windows;
 * Segment Length: 8.00 Seconds;
 * 0.5 Hz => DFT of 2 Seconds;
    
4. Welches Method: 1 Second DFT overlap;

5. Training: Segment overlap [%] = 96.875

6. Testing: Segment overlap [%] = 50%

## PSD and CSD analysis

In [ ]:
# 2.0: Butterworth
print("Finding pass-band frequencies via Cross Spectral Density Analysis")
# Find band pass values per channel

for i in range(0,21):
    plt.csd(X[:,i], y, Fs = 4096)
    plt.title("CSD over Channel i and Strain")

plt.xscale("log")
plt.show()


print("Finding pass-band frequencies via Cross Spectral Density Analysis")
## Find band pass values per channel
for i in range(0,21):
    plt.psd(X[:,i], Fs = 4096)
    plt.title("PSD over all Channels")

plt.xscale("log")
plt.show()

In [ ]:
print("Building Butterworth Filter")
N = 8 # order of butterworth filter

Wn = [0, 0.3] # Band-pass [low end, high end]
# Wn = [10, 100] # Band-pass [low end, high end]

filter_type = "bp"
fs = Wn[1]/2

b, a = signal.butter(N, Wn, filter_type, analog=True)
w, h = signal.freqs(b, a)
plt.semilogx(w, 20 * np.log10(abs(h)))
plt.title('Butterworth filter frequency response')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(100, color='green') # cutoff frequency
plt.show()
butter_sos = signal.butter(N, Wn, filter_type, fs , output = 'sos')
print("Butterworth filter built")

In [ ]:
# 2.1: Apply butter_sos to data
print("Applying Butterworth to data")
print(witness_data.shape)
plt.plot(witness_data[0][:,0])
plt.show()
for idx in range(len(witness_data)):
    for channel in range(channels):
        witness_data[idx][:, channel] = signal.sosfilt(butter_sos, witness_data[idx][:, channel])
plt.plot(witness_data[0][:,0])
plt.show()
print("Butterworth filter applied to data")

## 3: Standard Scalar (Z-score)

In [ ]:


print("Normalization in progress")

plt.plot(witness_data[0][:,0])
plt.show()

scaler = StandardScaler()
strain_data = np.reshape(strain_data, (-1,1))

## For Post Processing
strain_mean = np.mean(strain_data)
strain_std = np.std(strain_data)


scaler.fit(strain_data)
strain_data = scaler.transform(strain_data)


test_1 = scaler.transform(np.reshape(witness_data[1][:,0], (-1,1)))
plt.plot(test_1, label = "Z_score")
plt.plot(witness_data[0][:,0], label = "OG Data")
plt.legend()
plt.show()

print("CNN Data Ready, Import witness_data")
print(witness_data.shape)

# for idx in range(len(witness_data)):
#     for channel in range(channels):
#         reshaped_witness_data = np.reshape(witness_data[idx][:, channel], (-1,1))
#         reshaped_witness_data = scaler.transform(reshaped_witness_data)
# print(witness_data.shape)


print("Normalization Complete")

## 4: Windowing Parameters

In [ ]:
# 4: Windowing
# Should match window_length from Batching cell
window_length = 2
DFT_length = 2
sample_rate = 4096
window_size = window_length * sample_rate
DFT_size = DFT_length * sample_rate
overlap_percent = 0.96875



## 5: Welch's Method and DFT

In [ ]:
# 5: Welches Method
print("Applying Welch's Method")
window_type = 'hann'
nperseg_train = window_size
noverlap_train = window_size*(1-overlap_percent)
nfft = DFT_size
new_data = []

for batch in range(len(witness_data)):
    batch_data = []
    for channel in range(channels):
        welch_data = signal.welch(witness_data[batch][:, channel], sample_rate, window = 'hann')
        batch_data.append(welch_data[1])
    batch_data = np.stack(batch_data, axis = -1)
    new_data.append(batch_data)
new_data = np.stack(new_data, axis = 0)

print("Welch's Method Complete")


## API Call to send data

In [ ]:
def send_freq_data():
    """
    new_data == Spectral density data data,
    strain_data == GW processed data
    
    Used for loss function computaiton
    """
    return witness_data, strain_data

def send_time_data():
    """
    witness_data == filtered, and z_scored aux channel data,
    strain_data == GW processed data
    
    Used for CNN training. Not split yet.
    """
    return witness_data, strain_data


def send_post_processing_params():
    """
    strain_mean == mean of data used for z-score
    strain_std = std dev used for z-score
    
    Will be needed in post processing. Call function to return the old mean and std dev to reverse z-score
    """
    return strain_mean, strain_std